In [ ]:
import sys

import matplotlib.pyplot as plt 
%matplotlib inline  
import numpy as np
import scipy.stats # for creating a simple dataset 
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import one_hot
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

stg_path = '../'

if stg_path not in sys.path:
    sys.path.append(stg_path)

from stg import STG

In [ ]:
w1 = torch.Tensor([[1, -3], [-2, 2], [-1, 0.5], [0.7, -0.3]])
b1= torch.Tensor([0.5, -0.2])
w2 = torch.Tensor([[0.5], [-0.2]])
b2 = torch.Tensor([0])


l1 = nn.Linear(w1.shape[0], w1.shape[1])
l1.weight = nn.Parameter(w1.t())
l1.bias = nn.Parameter(b1)
l2 = nn.Linear(w2.shape[0], w2.shape[1])
l2.weight = nn.Parameter(w2.t())
l2.bias = nn.Parameter(b2)

simple_net = nn.Sequential(l1, nn.ReLU(), l2)

In [ ]:
test_net = nn.Sequential(nn.Linear(w1.shape[0], w1.shape[1]), nn.ReLU(), nn.Linear(w2.shape[0], w2.shape[1]))

In [ ]:
data = torch.randn(2000,4)
train_loader = DataLoader(TensorDataset(data), batch_size=32)

In [ ]:
epochs = 100
lr = 0.1
wd = 1e-8
skip_print=10

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(test_net.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
total_loss = 0


for e in range(epochs):
    ll = ((simple_net[0].weight - test_net[0].weight)**2).sum() + ((simple_net[2].weight - test_net[2].weight)**2).sum() 
    if e%skip_print ==0:
        print(ll.item(), total_loss)
    total_loss = 0
    for d in train_loader:
        dat = d[0]
        y = simple_net(dat)
        out = test_net(dat)
        loss = criterion(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        


In [ ]:
simple_net[0].weight

In [ ]:
test_net[0].weight

In [ ]:
simple_net[2].weight

In [ ]:
test_net[2].weight

In [ ]:
simple_net(data[5]).item(), test_net(data[5]).item()